# Creating a PINN to simulation 2D incompressible Navier-Stokes

We will taken training data from OpenFOAM tutorial.

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import Ofpp as ofpp
import matplotlib.animation as animation
import os

In [65]:
#Read in key parameters from control dict eventually
start_time = 0
end_time = 5
dt = 0.005
ofoam_write_interval = 1
ofoam_output_inc = dt * ofoam_write_interval
ofoam_num_outputs = ( end_time / ofoam_output_inc ) + 1

openfoam_case_path = "/Users/harry/openfoam/cavity-re100/"

x_start = 0
x_end = 1
y_start = 0
y_end = 1
numx = 20
numy = 20
numt = len(np.arange(start_time,end_time + ofoam_output_inc,ofoam_output_inc))
shape = (numx,numy,numt)
print(f"OpenFOAM data shape: {shape}")

print(f"Start: {start_time} end: {end_time}")
print(f"dt: {dt} num timesteps: {numt}")
print(f"Write interval: {ofoam_write_interval} ofoam_num_outputs: {ofoam_num_outputs}")
#print(f"Animation interval: {} animation images: {}")

X, Y = np.meshgrid(np.linspace(x_start,x_end,numx),
                   np.linspace(y_start,y_end,numy))

u_full = np.empty(shape)
v_full = np.empty(shape)
w_full = np.empty(shape)
p_full = np.empty(shape)

openfoam_outputs = os.listdir(openfoam_case_path)
openfoam_outputs.remove("constant")
openfoam_outputs.remove("system")
openfoam_outputs.remove("0_orig")
openfoam_outputs.sort()
openfoam_outputs = openfoam_outputs[::ofoam_write_interval]

idx = 0
for time in openfoam_outputs:
    openfoam_timestep = openfoam_case_path + time
    if time == "0":
        u_full[:-1,:,idx] = 0
        u_full[-1,:,idx] = 1
        
        v_full[:,:,idx] = 0
        
        p_full[:,:,idx] = 0
    else:
        U = ofpp.parse_internal_field(f"{openfoam_timestep}/U")
        u_full[:,:,idx] = U[:,0].reshape((numx,numy))
        v_full[:,:,idx] = U[:,1].reshape((numx,numy))
        
        p_full[:,:,idx] = ofpp.parse_internal_field(f"{openfoam_timestep}/p").reshape((numx,numy))
    
    idx += 1


OpenFOAM data shape: (20, 20, 1001)
Start: 0 end: 5
dt: 0.005 num timesteps: 1001
Write interval: 1 ofoam_num_outputs: 1001.0


In [66]:
u_full[:,1,0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1.])

In [39]:
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

/Users/harry/miniconda3/envs/pytorch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [40]:
#Use GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [67]:
_, _, grid = np.meshgrid(np.linspace(x_start,x_end,numx),
                         np.linspace(y_start,y_end,numy),
                         np.linspace(start_time,end_time,numt))

grid.shape

(20, 20, 1001)

In [90]:
 a, b, c = np.meshgrid(np.linspace(0,2,3),
                       np.linspace(0,2,3),
                       np.linspace(0,2,3))

a

array([[[0., 0., 0.],
        [1., 1., 1.],
        [2., 2., 2.]],

       [[0., 0., 0.],
        [1., 1., 1.],
        [2., 2., 2.]],

       [[0., 0., 0.],
        [1., 1., 1.],
        [2., 2., 2.]]])

In [91]:
b

array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.]]])

In [92]:
c

array([[[0., 1., 2.],
        [0., 1., 2.],
        [0., 1., 2.]],

       [[0., 1., 2.],
        [0., 1., 2.],
        [0., 1., 2.]],

       [[0., 1., 2.],
        [0., 1., 2.],
        [0., 1., 2.]]])

In [98]:
positions = np.vstack([a.ravel(), b.ravel(), c.ravel()])
positions

array([[0., 0., 0., 1., 1., 1., 2., 2., 2., 0., 0., 0., 1., 1., 1., 2.,
        2., 2., 0., 0., 0., 1., 1., 1., 2., 2., 2.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
       [0., 1., 2., 0., 1., 2., 0., 1., 2., 0., 1., 2., 0., 1., 2., 0.,
        1., 2., 0., 1., 2., 0., 1., 2., 0., 1., 2.]])

In [41]:
#Data wrangling to get 1D tensors for X,Y,t,u,v,p for interior and boundary conditions
#Convert to correct format
#Scaling
#Train-test split
#QC of train-test split
#Define NN structure

In [ ]:
# Define MLP model architecture class
class NeuralNetwork(torch.nn.Module):
    def __init__(self):
        """_summary_
        """
        super().__init__()
        self.linear_stack = torch.nn.Sequential(
            torch.nn.Linear(3, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 3)
        )

    def forward(self, x, y, t):
        """_summary_

        Args:
            x (_type_): _description_
            t (_type_): _description_

        Returns:
            _type_: _description_
        """
        inputs = torch.cat([x,y,t], axis=1).to(device)
        return self.linear_stack(inputs)

In [43]:
#Training loop

In [44]:
#Testing loop

In [45]:
#Loss function

In [46]:
#Main function

In [47]:
#QC plot and GIF using NN to predict fluid flow